In [ ]:
import os
import random
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [ ]:
############# Data Augmentation ####################


# Define the parent directory where "Drowsy" and "Non Drowsy" directories are located
parent_dir = 'C:\\Users\\USER\\Downloads\\DDD\\row'

# Define the output directories for augmented images
output_drowsy_dir = os.path.join('C:\\Users\\USER\\Downloads\\DDD\\augmented', 'Drowsy')
output_non_drowsy_dir = os.path.join('C:\\Users\\USER\\Downloads\\DDD\\augmented', 'Non Drowsy')

# Create output directories if they don't exist
os.makedirs(output_drowsy_dir, exist_ok=True)
os.makedirs(output_non_drowsy_dir, exist_ok=True)

# Create an instance of the ImageDataGenerator with augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=20,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    #shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Iterate through the "Drowsy" and "Non Drowsy" directories
classes = ["Drowsy", "Non Drowsy"]

for class_name in classes:
    class_dir = os.path.join(parent_dir, class_name)
    output_class_dir = output_drowsy_dir if class_name == "Drowsy" else output_non_drowsy_dir
    
    # List all files in the class directory
    image_files = [f for f in os.listdir(class_dir) if f.endswith('.jpg') or f.endswith('.png')]
    
    for image_file in image_files:
        img = load_img(os.path.join(class_dir, image_file))
        img = img_to_array(img)
        img = img.reshape((1,) + img.shape)
        
        # Generate augmented images
        i = 0
        for batch in datagen.flow(img, batch_size=1):
            augmented_image = batch[0].astype(np.uint8)
            
            # Save the augmented image to the output directory using PIL
            output_file = os.path.join(output_class_dir, f"augmented_{i}_{image_file}")
            i += 1
            augmented_image_pil = Image.fromarray(augmented_image)
            augmented_image_pil.save(output_file)
            
            # Limit the number of augmented images per input image (adjust as needed)
            if i >= 4:
                break

print("Data augmentation and saving completed.")

In [ ]:
######################## Resize Images #########################

# Define the input directory where augmented images are located
input_drowsy_dir = 'C:\\Users\\USER\\Downloads\\DDD\\augmented\\Drowsy'
input_non_drowsy_dir = 'C:\\Users\\USER\\Downloads\\DDD\\augmented\\Non Drowsy'

# Define the output directory for resized images
output_drowsy_dir_resized = 'C:\\Users\\USER\\Downloads\\DDD\\resize\\Drowsy'
output_non_drowsy_dir_resized = 'C:\\Users\\USER\\Downloads\\DDD\\resize\\Non Drowsy'

# Create output directories if they don't exist
os.makedirs(output_drowsy_dir_resized, exist_ok=True)
os.makedirs(output_non_drowsy_dir_resized, exist_ok=True)

# Define the target size (227x227)
target_size = (227, 227)

# Function to resize and pad an image to the target size
def resize_and_pad_image(input_path, output_path, target_size):
    img = Image.open(input_path)
    img = img.resize(target_size, Image.ANTIALIAS)
    
    # Create a new image with a white background
    new_img = Image.new("RGB", target_size, (255, 255, 255))
    
    # Paste the resized image onto the new image with center alignment
    x_offset = (target_size[0] - img.width) // 2
    y_offset = (target_size[1] - img.height) // 2
    new_img.paste(img, (x_offset, y_offset))
    
    new_img.save(output_path)

# Process "Drowsy" images
for filename in os.listdir(input_drowsy_dir):
    input_image_path = os.path.join(input_drowsy_dir, filename)
    output_image_path = os.path.join(output_drowsy_dir_resized, filename)
    resize_and_pad_image(input_image_path, output_image_path, target_size)

# Process "Non Drowsy" images
for filename in os.listdir(input_non_drowsy_dir):
    input_image_path = os.path.join(input_non_drowsy_dir, filename)
    output_image_path = os.path.join(output_non_drowsy_dir_resized, filename)
    resize_and_pad_image(input_image_path, output_image_path, target_size)

print("Image resizing and padding completed.")

In [ ]:
######################## Image Normalization #########################


# Define the input directory where images are located
input_directory = 'C:\\Users\\USER\\Downloads\\DDD\\resize'  # Replace with the actual path

# Define the output directories for normalized images
output_drowsy_dir_normalized = 'C:\\Users\\USER\\Downloads\\DDD\\normalized\\Drowsy'  # Replace with the actual path
output_non_drowsy_dir_normalized = 'C:\\Users\\USER\\Downloads\\DDD\\normalized\\Non Drowsy'  # Replace with the actual path

# Create output directories if they don't exist
os.makedirs(output_drowsy_dir_normalized, exist_ok=True)
os.makedirs(output_non_drowsy_dir_normalized, exist_ok=True)

# Define the target pixel value range (e.g., [0, 1])
min_value = 0
max_value = 1

# Function to normalize an image and save it
def normalize_and_save_image(input_path, output_path, min_value, max_value):
    img = Image.open(input_path)
    img_array = np.array(img)
    
    # Normalize pixel values to the specified range
    normalized_img_array = (img_array - np.min(img_array)) / (np.max(img_array) - np.min(img_array))
    normalized_img_array = (max_value - min_value) * normalized_img_array + min_value
    
    # Convert back to 8-bit integer format
    normalized_img_array = (normalized_img_array * 255).astype(np.uint8)
    
    normalized_img = Image.fromarray(normalized_img_array)
    normalized_img.save(output_path)

# Process "Drowsy" images
for filename in os.listdir(os.path.join(input_directory, "Drowsy")):
    input_image_path = os.path.join(input_directory, "Drowsy", filename)
    output_image_path = os.path.join(output_drowsy_dir_normalized, filename)
    normalize_and_save_image(input_image_path, output_image_path, min_value, max_value)

# Process "Non Drowsy" images
for filename in os.listdir(os.path.join(input_directory, "Non Drowsy")):
    input_image_path = os.path.join(input_directory, "Non Drowsy", filename)
    output_image_path = os.path.join(output_non_drowsy_dir_normalized, filename)
    normalize_and_save_image(input_image_path, output_image_path, min_value, max_value)

print("Image normalization completed.")